In [76]:
import pandas as pd
import json
import plotly.graph_objects as go

df = pd.read_csv("202501_202501_주민등록인구및세대현황_월간.csv", encoding='cp949')

geojson = json.load(open("법정구역_시군구.geojson", 'r', encoding="utf8"))

In [77]:
df['2025년01월_총인구수'] = df['2025년01월_총인구수'].apply(lambda x: int(x.replace(",","")))

def split_code_name(x):
    area_name, area_code = x.split('(')
    area_name = area_name.split()
    if len(area_name) == 1:
        area_name.append(area_name[0])
    sido, *area = area_name
    if sido[-2:] in ['북도', '남도']:
        sido = sido[::2]
    else:
        sido = sido[:2]
    return pd.Series([sido, " ".join(area), area_code[:5]])

df[['시도명','구역명','구역코드']] = df['행정구역'].apply(lambda x: split_code_name(x))

In [78]:
df[['시도명','구역명','구역코드']]

,시도명,구역명,구역코드
0,서울,서울특별시,11000
1,서울,종로구,11110
2,서울,중구,11140
3,서울,용산구,11170
4,서울,성동구,11200
...,...,...,...
287,경남,거창군,48880
288,경남,합천군,48890
289,제주,제주특별자치도,50000
290,제주,제주시,50110


In [79]:
sido_goejson = {feature['properties']['SIG_CD']: feature["properties"]['SIG_KOR_NM'] for feature in geojson["features"]}
df['구역명 확인'] = df['구역코드'].map(sido_goejson)
df = df[df['구역명 확인'].notna()].reset_index(drop=True)

df['구역명 확인'].value_counts()

구역명 확인
중구      6
동구      6
서구      5
남구      4
북구      4
       ..
함양군     1
거창군     1
합천군     1
제주시     1
서귀포시    1
Name: count, Length: 230, dtype: int64

In [80]:
fig = go.Figure(
    data= go.Choropleth(
        geojson=geojson,
        featureidkey='properties.SIG_CD',
        locationmode='geojson-id',
        locations=df['구역코드'],
        z=df['2025년01월_총인구수'],
        colorscale=[[0,'#F1EFEF'], [0.2, '#CCCBAA'], 
                    [0.4, '#B0AA7C'], [0.6, '#7D7C7C'], 
                    [0.8, '#606068'], [1, '#191717']],
        hovertext = df['구역명']+": "+df['2025년01월_총인구수'].apply(lambda x: f"{x:,} 명"),
        hoverinfo='text',
        marker_line_color="#050505",
        marker_line_width=1
    )
)

fig.update_layout(
    geo=dict(fitbounds='locations',
    visible=False,
    showframe=False,
    ),
    margin={'r':0, 't':0, "l":0, 'b':0}
)

html = fig.to_html()
with open('시군구별 인구현황_202501.hml','w',encoding='utf8') as f:
    f.write(html) 